In [1]:
#hide
#skip
%config Completer.use_jedi = False
%config IPCompleter.greedy=True
# upgrade fastrl on colab
! [ -e /content ] && pip install -Uqq fastrl['dev'] pyvirtualdisplay && \
                     apt-get install -y xvfb python-opengl > /dev/null 2>&1 
# NOTE: IF YOU SEE VERSION ERRORS, IT IS SAFE TO IGNORE THEM. COLAB IS BEHIND IN SOME OF THE PACKAGE VERSIONS

In [2]:
# hide
from fastcore.imports import in_colab
# Since colab still requires tornado<6, we don't want to import nbdev if we don't have to
if not in_colab():
    from nbdev.showdoc import *
    from nbdev.imports import *
    if not os.environ.get("IN_TEST", None):
        assert IN_NOTEBOOK
        assert not IN_COLAB
        assert IN_IPYTHON
else:
    # Virutual display is needed for colab
    from pyvirtualdisplay import Display
    display = Display(visible=0, size=(400, 300))
    display.start()

In [3]:
# default_exp envs.gym

In [4]:
# export
# Python native modules
import os
# Third party libs
from fastcore.all import *
import torchdata.datapipes as dp
import gym
from fastai.torch_basics import *
from fastai.torch_core import *
# Local modules
from fastrl.fastai.data.pipes.core import *
from fastrl.fastai.data.load import *
from fastrl.fastai.data.block import *
from fastrl.envs.core import *

# Envs Gym
> Fastrl API for working with OpenAI Gyms

### Pipes

In [ ]:
dp.iter.IterDataPipe

In [5]:
# export
class GymTypeTransform(Transform):
    def encodes(self,o): return gym.make(o)

class GymStepper(dp.iter.IterDataPipe):
    def __init__(self,source_datapipe,agent=None):
        self.source_datapipe = source_datapipe
        self.agent = agent
        self._env_ids = {}
        
    def __iter__(self):
        for env in self.source_datapipe:
            env_id = id(env)
            
            
            
            

        if getattr(o,'is_done',True):
            state = o.reset(seed=getattr(self,'seed',0))
            o.is_done = False
            o.sum_reward = 0
            episode_n = getattr(o,'step_info',{'episode_n':-1})['episode_n']+1
            o.step_info = make_step(state,None,False,None,0,None,env_id=id(o),step_n=-1,
                                    episode_n=episode_n)
        else:
            state = o.state
            
        self.agent.agent_base.iterator.append(o.step_info)
        for action in self.agent:
            next_state,reward,done,_ = o.step(action)
            o.sum_reward += reward
        
        if done: o.is_done = True
        o.step_info = make_step(state,next_state,done,reward,o.sum_reward,action,env_id=id(o),
                                step_n=o.step_info['step_n']+1,episode_n=o.step_info['episode_n'])
        
        return o.step_info
            
            

In [39]:
pipe = dp.map.Mapper(['CartPole-v1']*3)
pipe = TypeTransformLoop(pipe,[GymTypeTransform])
pipe = dp.iter.MapToIterConverter(pipe)
pipe = pipe.cycle()

In [43]:
from torch.utils.data.dataloader_experimental import DataLoader2

def seed_worker(worker_id): torch.manual_seed(0)

dl = DataLoader2(pipe,num_workers=2,worker_init_fn=seed_worker)

for o in dl:
    print(o)
    break

/opt/conda/lib/python3.7/site-packages/gym/utils/passive_env_checker.py:98: UserWarning: WARN: We recommend you to use a symmetric and normalized Box action space (range=[-1, 1]) https://stable-baselines3.readthedocs.io/en/master/guide/rl_tips.html
  "We recommend you to use a symmetric and normalized Box action space (range=[-1, 1]) "


TypeError: Caught TypeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/torch/utils/data/_utils/worker.py", line 302, in _worker_loop
    data = fetcher.fetch(index)
  File "/opt/conda/lib/python3.7/site-packages/torch/utils/data/_utils/fetch.py", line 40, in fetch
    return self.collate_fn(data)
  File "/opt/conda/lib/python3.7/site-packages/torch/utils/data/_utils/collate.py", line 183, in default_collate
    raise TypeError(default_collate_err_msg_format.format(elem_type))
TypeError: default_collate: batch must contain tensors, numpy arrays, numbers, dicts or lists; found <class 'gym.wrappers.time_limit.TimeLimit'>


/opt/conda/lib/python3.7/site-packages/gym/utils/passive_env_checker.py:98: UserWarning: WARN: We recommend you to use a symmetric and normalized Box action space (range=[-1, 1]) https://stable-baselines3.readthedocs.io/en/master/guide/rl_tips.html
  "We recommend you to use a symmetric and normalized Box action space (range=[-1, 1]) "


In [ ]:
# hide
from fastcore.imports import in_colab

# Since colab still requires tornado<6, we don't want to import nbdev if we don't have to
if not in_colab():
    from nbdev.export import *
    from nbdev.export2html import *
    from nbdev.cli import *
    make_readme()
    notebook2script(silent=True)